**Import all the libraries :**

In [ ]:
import os
import gc
import json
import pickle
import zipfile
import warnings 
import datetime
import lightgbm
import prettytable
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from tqdm import tqdm
sns.set_style("whitegrid")
from functools import reduce
import matplotlib.pylab as plt
warnings.filterwarnings('ignore')
from IPython.display import Image
from sklearn import preprocessing
from prettytable import PrettyTable
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer
from scipy.stats import randint as sp_randint
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsRegressor

**Loading all Functions :**

In [ ]:
def loadData():


  train_data       = pd.read_csv('/content/train.csv', parse_dates=["first_active_month"])
  test_data        = pd.read_csv('/content/test.csv', parse_dates=["first_active_month"])
  historical_data  = pd.read_csv('/content/historical_transactions.csv',parse_dates=['purchase_date'])
  newmerchant_data = pd.read_csv('/content/new_merchant_transactions.csv',parse_dates=["purchase_date"])
  
  return train_data, test_data, newmerchant_data, historical_data 

In [ ]:
# Reference: https://www.kaggle.com/rinnqd/reduce-memory-usage

def reduce_memory_usage(df, verbose=True):
  '''
  The data size is too big to get rid of memory error this method will reduce memory
  usage by changing types. It does the following
  - Load objects as categories
  - Binary values are switched to int8
  - Binary values with missing values are switched to float16
  - 64 bits encoding are all switched to 32 or 16bits if possible.
  
  Parameters :
  df - DataFrame whose size to be reduced
  verbose - Boolean, to mention the verbose required or not.
  '''
  numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  start_mem = df.memory_usage().sum() / 1024**2
  for col in df.columns:
      col_type = df[col].dtypes
      if col_type in numerics:
          c_min = df[col].min()
          c_max = df[col].max()
          if str(col_type)[:3] == 'int':
              if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                  df[col] = df[col].astype(np.int8)
              elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                  df[col] = df[col].astype(np.int16)
              elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                  df[col] = df[col].astype(np.int32)
              elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                  df[col] = df[col].astype(np.int64)
          else:
              c_prec = df[col].apply(lambda x: np.finfo(x).precision).max()
              if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max and c_prec == np.finfo(np.float16).precision:
                  df[col] = df[col].astype(np.float16)
              elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max and c_prec == np.finfo(np.float32).precision:
                  df[col] = df[col].astype(np.float32)
              else:
                  df[col] = df[col].astype(np.float64)
  end_mem = df.memory_usage().sum() / 1024**2
  if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
  return df

In [ ]:
def baseline_features(train_data,test_data,historical_data,newmerchant_data):
   '''This function is for adding basic features
      on the data.
      parameters:
      - train_data: dataframe for train.csv
      - test_data: dataframe for test.csv
      - historical_data: dataframe for historical_transactions.csv
      - newmerchant_data: dataframe for new_merchant_transaction.csv'''

  #1. Transform first_active_month and extract year and month :

  # In train_data
  train_data['first_active_month'] = pd.to_datetime(train_data['first_active_month'])

  # In test_data
  test_data['first_active_month'] = pd.to_datetime(test_data['first_active_month'])

  for df in [train_data, test_data]:
    # extracting the year and month
    df['first_active_year'] = df['first_active_month'].dt.year.values
    df['first_active_mon'] = df['first_active_month'].dt.month.values

  # Encode first_active_year column
  le = preprocessing.LabelEncoder()
  train_data['first_active_year'] = le.fit_transform(train_data['first_active_year'] )
  test_data['first_active_year'] = le.fit_transform(test_data['first_active_year'] )

  # Encode first_active_mon column
  train_data['first_active_mon'] = le.fit_transform(train_data['first_active_mon'] )
  test_data['first_active_mon'] = le.fit_transform(test_data['first_active_mon'] )

  #2. Derive last purchase amount, last active month and dormancy feature :

  ## For historical transaction
  historical_data['purchase_date'] = pd.to_datetime(historical_data['purchase_date'])

  # last active month & last purchase amount
  last_active_month = historical_data.loc[historical_data.groupby('card_id').purchase_date.idxmax(),:][['card_id','purchase_date','purchase_amount']]
  last_active_month.columns = ['card_id','hist_transc_last_active_purchase_date','hist_transc_last_active_purchase_amount']
  train_data = pd.merge(train_data,last_active_month, on="card_id",how='left')
  test_data = pd.merge(test_data,last_active_month, on="card_id",how='left')

  train_data['hist_transc_last_active_purchase_month'] = train_data['hist_transc_last_active_purchase_date'].dt.month
  test_data['hist_transc_last_active_purchase_month'] = test_data['hist_transc_last_active_purchase_date'].dt.month

  # dormancy feature
  max_purchase_date_hist = historical_data['purchase_date'].max()
  train_data['hist_transc_dormancy'] = [(max_purchase_date_hist-x).days for x in train_data['hist_transc_last_active_purchase_date']]
  test_data['hist_transc_dormancy'] = [(max_purchase_date_hist-x).days for x in test_data['hist_transc_last_active_purchase_date']]
  train_data.head()

  ## For new_merchant_transaction
  newmerchant_data['purchase_date'] = pd.to_datetime(newmerchant_data['purchase_date'])

  # last active month & last purchase amount
  last_active_month = newmerchant_data.loc[newmerchant_data.groupby('card_id').purchase_date.idxmax(),:][['card_id','purchase_date','purchase_amount']]
  last_active_month.columns = ['card_id','new_transc_last_active_purchase_date','new_transc_last_active_purchase_amount']
  train_data = pd.merge(train_data, last_active_month, on="card_id",how='left')
  test_data = pd.merge(test_data, last_active_month, on="card_id",how='left')

  train_data['new_transc_last_active_purchase_month'] = train_data['new_transc_last_active_purchase_date'].dt.month
  test_data['new_transc_last_active_purchase_month'] = test_data['new_transc_last_active_purchase_date'].dt.month

  # dormancy feature
  max_purchase_date_newmer = newmerchant_data['purchase_date'].max()
  train_data['new_transc_dormancy'] = [(max_purchase_date_newmer-x).days for x in train_data['new_transc_last_active_purchase_date']]
  test_data['new_transc_dormancy'] = [(max_purchase_date_newmer-x).days for x in test_data['new_transc_last_active_purchase_date']]

  #3. Deriving Favourite merchant and Number of transactions at Favourite merchant as feature :

  # For historical transaction
  merchantid_counts_percard = pd.DataFrame(historical_data[['card_id','merchant_id']].groupby(['card_id','merchant_id']).size())
  merchantid_counts_percard.columns = ['num_favourite_merchant']
  merchantid_counts_percard = merchantid_counts_percard.sort_values(by='num_favourite_merchant',ascending=False)
  merchantid_counts_percard = merchantid_counts_percard.groupby(level=0).head(1).reset_index()
  merchantid_counts_percard.columns = ['card_id','hist_transc_favourite_merchant','hist_transc_num_transaction_favourite_merchant']
  train_data = pd.merge(train_data ,merchantid_counts_percard, on="card_id",how='left')
  test_data = pd.merge(test_data ,merchantid_counts_percard, on="card_id",how='left')

  # Encode Favourite merchant column
  train_data['hist_transc_favourite_merchant'] = le.fit_transform(train_data['hist_transc_favourite_merchant'] )
  test_data['hist_transc_favourite_merchant'] = le.fit_transform(test_data['hist_transc_favourite_merchant'] )

  # For new_merchant_transaction
  merchantid_counts_percard = pd.DataFrame(newmerchant_data[['card_id','merchant_id']].groupby(['card_id','merchant_id']).size())
  merchantid_counts_percard.columns = ['num_favourite_merchant']
  merchantid_counts_percard = merchantid_counts_percard.sort_values(by='num_favourite_merchant',ascending=False)
  merchantid_counts_percard = merchantid_counts_percard.groupby(level=0).head(1).reset_index()
  merchantid_counts_percard.columns = ['card_id','new_transc_favourite_merchant','new_transc_num_transaction_favourite_merchant']
  train_data = pd.merge(train_data ,merchantid_counts_percard, on="card_id",how='left')
  test_data = pd.merge(test_data ,merchantid_counts_percard, on="card_id",how='left')

  train_data['new_transc_favourite_merchant'].fillna('M_ID_00a6ca8a8a',inplace=True)
  test_data['new_transc_favourite_merchant'].fillna('M_ID_00a6ca8a8a',inplace=True)

  # Encode Favourite merchant column
  train_data['new_transc_favourite_merchant'] = le.fit_transform(train_data['new_transc_favourite_merchant'] )
  test_data['new_transc_favourite_merchant'] = le.fit_transform(test_data['new_transc_favourite_merchant'] )

  return train_data, test_data

In [ ]:
def imputation(df_hist, df_new):
  '''This function is for missing value imputation in data
     parameters:
     - df_test: test_data
     - df_hist: historical_data
     - df_new: newmerch_data.'''

  # In historical_data
  df_hist['category_2'].fillna(1,inplace=True)# I put '1' here because it is most occured value in this feature
  df_hist['category_3'].fillna('A',inplace=True)# I put 'A' here because of most count value
  df_hist['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)# same merchant_id is also been taken on the basis of count.

  # I use same techniques for newmerchant_data
  df_new['category_3'].fillna('A',inplace=True)
  df_new['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True)
  df_new['category_2'].fillna(1.0,inplace=True)

  return df_hist, df_new

In [ ]:
def encode_categorical(df):
  '''This function is specially for encode the categorical values of 
     transactions data
     parameters:
     - df: the Dataframe where the label encoding will performed on certain features'''
  
  ## label encode the categorical variables.
  e = {'N':0, 'Y':1}
  df['category_1']= df['category_1'].map(e)
  df['authorized_flag']= df['authorized_flag'].map(e)
  return df

In [ ]:
def oneHotEncoding(df, features, original_df):
  '''This function is for one-hot encoding the categorical features
     parameters:
     - df: DataFrame
     - features: Features needs to be one hot encoded.'''

  for feat in features:
    unique_values = original_df[feat].unique()

    for cat in unique_values:
      df[feat+'={}'.format(cat)] = (df[feat] == cat).astype(int)

In [ ]:
def aggregated_features(new_df, df, aggs, grpby, name='',  prefix='', use_col=False):
  '''
  This function is to find the 
  aggregated values (sum,min,max,std,median,mean,nunique) for a columns aggregated by the groupby operation
  
  Parameters:
  new_df   - features will be added to this DF
  df       - original DF from which the features will be created
  grpby    - based on this column we'll to group by
  name     - name for the new features created
  aggs     - dictionary contains key as the column the operation performed and list of operations as the value.
  prefix   - added to the name of the feature -- default value empty
  use_col  - if set True then the original column name will be uesd to name the new feature -- default value False
  '''
  # boolean for using the original column name in the aggregated features
  # iterating through the columns of the need to be aggregated 

  for col, funcs in aggs.items():
    for func in funcs:
        # Getting the name of aggregation function
        if isinstance(func, str):
            func_str = func
        else:
            func_str = func.__name__ 
        # create the column
        if use_col:
          name = prefix+'_'+col+'_'+'{}'.format(func_str)

        new_df[name] = df.groupby([grpby])[col].agg(func).values

  return new_df

In [ ]:
def data_preprocessing(historical_data,newmerchant_data):
  '''This function is for performing preprocessing 
  practices on the data.
  parameters:
  historical_data: data from historical_transaction
  newmerchant_data: data from nemerch_transaction'''
    
  ## imputing the missing values
  print(' - Imputing Missing values...')
  historical_data, newmerchant_data = imputation(historical_data,newmerchant_data)
  
  ## encoding the categorical features in historical transactions
  historical_data = encode_categorical(historical_data)

  ## encoding the categorical features in new_merchants
  newmerchant_data = encode_categorical(newmerchant_data)

  ## One-hot encoding the categorical features
  categorical_features = ['category_2','category_3','month_lag']

  ## one-hot encoding historical transactions
  print(' - One Hot Encoding of variables...')
  oneHotEncoding(historical_data, features=categorical_features, original_df = historical_df)

  ## one-hot encoding new merchants transactions
  oneHotEncoding(newmerchant_data, features=categorical_features, original_df = newmerchant_df)
  

  ## calcuating month difference
  reference_date = '2018-12-31'
  reference_date = pd.to_datetime(reference_date)

  # In historical_transactions 
  historical_data['month_diff'] = (reference_date - historical_data['purchase_date']).dt.days // (30 + historical_data['month_lag'])
  # In new_merch_transactions
  newmerchant_data['month_diff'] = (reference_date - newmerchant_data['purchase_date']).dt.days // (30 + newmerchant_data['month_lag'])

  ## Influential days feature
  holidays = {'FathersDay_2017': '2017-08-13',
              'ChildrenDay_2017':'2017-10-12',
              'BlackFriday_2017':'2017-11-24',
              'ValentineDay_2017':'2017-06-12',
              'Republicday_2017':'2017-11-15',
              'Independenceday_2017':'2017-09-7',
              'EasterDay_2017' : '2017-04-16',
              'AllSoulsDay_2017': '2017-11-2',
              'ChristmasDay_2017': '2017-12-25'}

  # In historical_transactions 
  for day, date in holidays.items():
    historical_data[day] = (pd.to_datetime(date) - historical_data['purchase_date']).dt.days
    historical_data[day] = historical_data[day].apply(lambda x: x if x > 0 and x < 15 else 0)
  # In new_transactions
  for day, date in holidays.items(): 
    newmerchant_data[day] = (pd.to_datetime(date) - newmerchant_data['purchase_date']).dt.days
    newmerchant_data[day] = newmerchant_data[day].apply(lambda x: x if x > 0 and x < 15 else 0)

  ## preprocess the purchase_amount
  newmerchant_data['purchase_amount'] = np.round(newmerchant_data['purchase_amount'] / 0.00150265118 + 497.06, 2)
  historical_data['purchase_amount'] = np.round(historical_data['purchase_amount'] / 0.00150265118 + 497.06, 2)

  ## The ratio of purchase amount by month_diff is another feature which help to reveal the card_id's financial capcity and purchase_pattern.
  # In historical_transactions 
  historical_data['amount_month_ratio'] = historical_data['purchase_amount'].values / (1.0 + historical_data['month_diff'].values)
  # In new_merch_transactions
  newmerchant_data['amount_month_ratio'] = newmerchant_data['purchase_amount'].values / (1.0 + newmerchant_data['month_diff'].values)

  ##is_weekend is a feature which purchase_date is weekend or weekday.
  ##greater than 5 to check whether the day is sat or sunday then, if it is then assign a val 1 else 0
  # In historical_transactions 
  historical_data['is_weekend'] = historical_data['purchase_date'].dt.dayofweek
  historical_data['is_weekend'] = historical_data['is_weekend'].apply(lambda x: 1 if x >= 5 else 0).values
  # In new_merch_transactions
  newmerchant_data['is_weekend'] = newmerchant_data['purchase_date'].dt.dayofweek
  newmerchant_data['is_weekend'] = newmerchant_data['is_weekend'].apply(lambda x: 1 if x >= 5 else 0).values

  ## extracting the day, hour , week from the purchase_date.
  # In historical_transactions
  historical_data['purchase_date_week'] = historical_data['purchase_date'].dt.week.values
  historical_data['purchase_date_dayofweek'] = historical_data['purchase_date'].dt.dayofweek.values
  historical_data['purchase_date_hour'] = historical_data['purchase_date'].dt.hour.values
  # In new_merch_transactions
  newmerchant_data['purchase_date_week'] = newmerchant_data['purchase_date'].dt.week.values
  newmerchant_data['purchase_date_dayofweek'] = newmerchant_data['purchase_date'].dt.dayofweek.values
  newmerchant_data['purchase_date_hour'] = newmerchant_data['purchase_date'].dt.hour.values

  return historical_data, newmerchant_data

In [ ]:
def feature_engineering(historical_data,newmerchant_data):
  '''This function is for performing feature engineering
     on the data.
     parameters:
     - historical_data: data from historical_transaction
     - newmerchant_data: data from nemerch_transaction'''
  
  # In historical_transactions 
  features_historical = pd.DataFrame(historical_data.groupby(['card_id']).size()).reset_index()
  features_historical.columns = ['card_id', 'hist_transc_count']
  # # In new_merch_transactions 
  features_newmerch = pd.DataFrame(newmerchant_data.groupby(['card_id']).size()).reset_index()
  features_newmerch.columns = ['card_id', 'new_transc_count']
  
  ## Aggregation all the id's
  aggs = {'city_id':['nunique'],
          'state_id' :['nunique'],
          'merchant_category_id':['nunique'],
          'subsector_id':['nunique'],
          'merchant_id':['nunique']}

  # In historical_transactions        
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  # In new_merch_transactions 
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)

  ## Aggregation Installment feature 
  aggs = {'installments':['mean', 'sum', 'max', 'min', 'std', 'skew']}

  # In historical_transactions        
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)

  ## Aggregation purchase_amount feature
  aggs = {'purchase_amount':['sum', 'mean', 'max', 'min', 'median', 'std', 'skew']}

  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id', prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id', prefix='new_transc', use_col=True)

  ## Aggregation month_lag feature 
  aggs = {'month_lag': ['nunique', 'mean', 'std', 'min', 'max', 'skew']}

  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)

  ## Aggregation purchase_date feature
  aggs = {'purchase_date': ['max','min']}

  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)

  ## Aggregation category_1 & authorized_flag features
  aggs = {'category_1':['sum', 'mean'],
        'authorized_flag': ['sum', 'mean']}

  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch,newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)

  ## Aggregation category_2 & category_3 features
  aggs = {'category_2=1.0':['sum', 'mean'],
          'category_2=2.0':['sum', 'mean'],
          'category_2=3.0':['sum', 'mean'],
          'category_2=4.0':['sum', 'mean'],
          'category_2=5.0':['sum', 'mean'],
          'category_3=1.0':['sum', 'mean'],
          'category_3=2.0':['sum', 'mean'],
          'category_3=3.0':['sum', 'mean']}

  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)
  
  ### Derived Features : from existing features

  ## Authorized_flag
  # historical_transactions
  features_historical['hist_transc_denied_count'] = features_historical['hist_transc_count'].values - \
                                                    features_historical['hist_transc_authorized_flag_sum'].values
  # new_merchant_transactions
  features_newmerch['new_transc_denied_count'] = features_newmerch['new_transc_count'].values - \
                                                    features_newmerch['new_transc_authorized_flag_sum'].values

  ## category_1
  # historical_transactions
  features_historical['hist_transc_category_1_sum_0'] = features_historical['hist_transc_count'].values - \
                                                              features_historical['hist_transc_category_1_sum'].values
  # new_merchant_transactions
  features_newmerch['new_transc_category_1_sum_0'] = features_newmerch['new_transc_count'].values - \
                                                            features_newmerch['new_transc_category_1_sum'].values
  
  ## find mean of the count of the transac for merchant id
  # historical_transactions
  features_historical['hist_transc_merchant_id_count_mean'] = features_historical['hist_transc_count'].values \
                                                                    / (1.0+features_historical['hist_transc_merchant_id_nunique'].values)
  # new_merchant_transactions
  features_newmerch['new_transc_merchant_id_count_mean'] = features_newmerch['new_transc_count'].values \
                                                              / (1.0+features_newmerch['new_transc_merchant_id_nunique'].values)
  
  ## In historical_transactions
  # diff in purchase_date from max to min 
  features_historical['hist_transc_purchase_date_diff'] = (features_historical['hist_transc_purchase_date_max'] - features_historical['hist_transc_purchase_date_min']).dt.days.values
  # purchase_count_ratio
  features_historical['hist_transc_purchase_count_ratio'] = features_historical['hist_transc_count'].values / (1.0 + features_historical['hist_transc_purchase_date_diff'].values)

  ## In new_merch_transactions
  # diff in purchase_date from max to min 
  features_newmerch['new_transc_purchase_date_diff'] = (features_newmerch['new_transc_purchase_date_max'] - features_newmerch['new_transc_purchase_date_min']).dt.days.values
  # purchase_count_ratio
  features_newmerch['new_transc_purchase_count_ratio'] = features_newmerch['new_transc_count'].values / (1.0 + features_newmerch['new_transc_purchase_date_diff'].values)

  ## aggregate features for is_weekend 
  aggs = {'is_weekend': ['sum','mean']}

  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='purchase_is_weekend_', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='purchase_is_weekend_', use_col=True)

  ## aggregated features on  day, hour , week
  aggs = {'purchase_date_week': ['nunique', 'mean', 'min', 'max'],
          'purchase_date_dayofweek': ['nunique', 'mean', 'min', 'max'],
          'purchase_date_hour':['nunique', 'mean', 'min', 'max']}

  # In historical_transactions
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)

  ## calculating the aggerated features like ['mean', 'min', 'max'] for month_diff column.
  aggs = {'month_diff': ['mean', 'min', 'max']}

  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id', prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id', prefix='new_transc', use_col=True)

  # aggregated features on the amount ratio and month_lag.
  aggs = {'amount_month_ratio': ['mean', 'std', 'min', 'max', 'skew'],
          'month_lag=0': ['sum','mean'],
          'month_lag=-1':['sum','mean'],
          'month_lag=-2':['sum','mean']}
  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)

  # aggregated features on the amount ratio and month_lag.
  aggs = {'amount_month_ratio': ['mean', 'std', 'min', 'max', 'skew'],
          'month_lag=1': ['sum','mean'],
          'month_lag=2':['sum','mean']}
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)

  # month_lag ratio in historical transactions
  features_historical['hist_transc_month_lag_0_-1_ratio'] = features_historical['hist_transc_month_lag=0_sum'] \
                                                                  / (1.0 + features_historical['hist_transc_month_lag=-1_sum'])

  features_historical['hist_transc_month_lag_0_-2_ratio'] = features_historical['hist_transc_month_lag=0_sum'] \
                                                                  / (1.0 + features_historical['hist_transc_month_lag=-2_sum'])

  # ratio of the summed month lags with the transaction count
  tmp = features_historical[['hist_transc_month_lag=0_sum','hist_transc_month_lag=-1_sum','hist_transc_month_lag=-2_sum']].sum(axis=1)

  features_historical['hist_transc_month_lag_sum_ratio'] = tmp / (1.0 + features_historical['hist_transc_count'])

  # month_lag ratio between two month_lags.
  features_newmerch['new_transc_month_lag_1_2_ratio'] = features_newmerch['new_transc_month_lag=1_sum'] \
                                                                  / (1.0 + features_newmerch['new_transc_month_lag=2_sum'])

  ## difference in the amount spend with cards
  # In historical_transactions 
  features_historical['hist_transc_amount_diff'] = features_historical['hist_transc_purchase_amount_max'].values - features_historical['hist_transc_purchase_amount_min'].values
  # In new_merch_transactions
  features_newmerch['new_transc_amount_diff'] = features_newmerch['new_transc_purchase_amount_max'].values - features_newmerch['new_transc_purchase_amount_min'].values

  ## Influential days feature
  holidays = {'FathersDay_2017': '2017-08-13',
              'ChildrenDay_2017':'2017-10-12',
              'BlackFriday_2017':'2017-11-24',
              'ValentineDay_2017':'2017-06-12',
              'Republicday_2017':'2017-11-15',
              'Independenceday_2017':'2017-09-7',
              'EasterDay_2017' : '2017-04-16',
              'AllSoulsDay_2017': '2017-11-2',
              'ChristmasDay_2017': '2017-12-25'}

  ## aggregation of holidays
  aggs = dict(zip(holidays.keys(),[['mean'] for x in holidays.keys()]))
  # In historical_transactions 
  features_historical = aggregated_features(features_historical, historical_data, aggs, grpby='card_id',prefix='hist_transc', use_col=True)
  # In new_merch_transactions
  features_newmerch = aggregated_features(features_newmerch, newmerchant_data, aggs, grpby='card_id',prefix='new_transc', use_col=True)
  
  return features_historical, features_newmerch


In [ ]:
def merge_drop(df,features_historical,features_newmerch):
   '''This function is for merging the engineered features and
      removing the unnecessary features.
      parameters:
      - df: data for prediction
      - features_historical: features from historical_transc
      - features_newmerch: features from nemerch_transc'''

  train = reduce(lambda left,right: pd.merge(left,right,on='card_id', how='left'), [df, features_historical, features_newmerch])
  
  remove_cols = ['first_active_month','new_transc_purchase_date_max',
 'new_transc_purchase_date_min','hist_transc_purchase_date_max',
 'hist_transc_purchase_date_min', 'hist_transc_last_active_purchase_date', 'new_transc_last_active_purchase_date']

  train = train.drop(labels=remove_cols, axis = 1)

  return train

In [ ]:
def root_mean_squared_error(X , y_true):
  '''This function is for calculating the RMSE score
  parameters:
  X: test_data
  y_true: original target score'''

  y_predicted = predict_loyalty_score(X)
  ## calculating the RMSE score
  rmse = np.sqrt(mean_squared_error(y_predicted, y_true))
  print('RMSE Score:', rmse)

  return rmse

In [ ]:
def predict_loyalty_score(X):
  ''' This function predict's loyalty Score of the given card_id/ids
  parameters:
  X : card_id/List of the card_ids
  returns Dataframe with predicted loyalty score for card_id/ids.'''
  

  ## training the data
  # Fetching the transactional and card_id data
  print("Fetching the transactional and card_id data")
  sample = cards.loc[cards['card_id'].isin(X)]
  newmerchant_data = newmerchant_df[newmerchant_df['card_id'].isin(X)]
  historical_data = historical_df[historical_df['card_id'].isin(X)]

  ## Performing preprocessing steps on transactions data
  print("PreProcess the transactions data......")
  historical_data, newmerchant_data = data_preprocessing(historical_data,newmerchant_data)

  ## Performing feature Engineering on transactions data
  print("Feature Engineering the transactions data.....")
  features_historical, features_newmerch = feature_engineering(historical_data,newmerchant_data)

  ## Performing preprocessing on Feature Engineered Data
  print('preprocess the Feature Engineered Data')
  query_point = merge_drop(sample,features_historical,features_newmerch)
  query_point.set_index('card_id', inplace=True)

  ## Predicting the Loyalty Score
  print("Predicting the Loyalty Score.....")
  with open('/content/drive/MyDrive/Case study1/xgb_model_final.sav', 'rb') as pickle_file:
      xgboost_model = pickle.load(pickle_file)
  with open('/content/drive/MyDrive/Case study1/lgbm_model1.sav', 'rb') as pickle_file:
    lgbm_model = pickle.load(pickle_file)
  with open('/content/drive/MyDrive/Case study1/stacked_model1.sav', 'rb') as pickle_file:
      stacked_model = pickle.load(pickle_file)

  xgboost_predictions = xgboost_model.predict(xgb.DMatrix(query_point[xgboost_model.feature_names]), ntree_limit=xgboost_model.best_ntree_limit+50)  
  lgbm_predictions = lgbm_model.predict(query_point, num_iteration=lgbm_model.best_iteration)
  stacked_prediction = np.vstack([xgboost_predictions, lgbm_predictions]).transpose()
  final_prediction = stacked_model.predict(stacked_prediction)

  ## Preparing the output
  Score_df = pd.DataFrame()
  Score_df['card_id'] = query_point.index 
  Score_df['loyality_score'] = final_prediction
  Score_df.set_index('card_id', inplace=True)
  
  return Score_df

**Downloading data :**

In [ ]:
!wget --header="Host: storage.googleapis.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9" --header="Accept-Language: en-US,en;q=0.9" --header="Referer: https://www.kaggle.com/" "https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10445/200747/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1620215211&Signature=Mt2Std3ijid8ntLwh3anT4DJ2t%2Ba4JzlrlBq29yXAqnwJlCLBWq3qenMV6Pr6QE3ZaWIiZfBFknZQ7EcMNJJtYS2l5GrnCRu68uT%2FPaZSSWoOA4PzOBFMAwnrffU8q%2FRogni4R0j%2B%2FFBGz%2FnrHw%2B2jLG%2F4%2F7gEY%2F7jhrNptqbVJFnLPxAwGWmlU9eGutByplN3A7gfSlbGk5TybkOPPzCG%2F%2B5TSvDmkPzkyNZ3wa84dbsjCLYiSvWFT0I%2FsnazgiVXUKWJUjF6AnQ3NNBYqWrWO2Kjc0s%2F63ZIhr06MvDdbe3C%2BJgCKiL8KV7vffGjyMhq4m1bGSdaD01VyZg0x19w%3D%3D&response-content-disposition=attachment%3B+filename%3Delo-merchant-category-recommendation.zip" -c -O 'elo-merchant-category-recommendation.zip'

--2021-05-03 09:52:38--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/10445/200747/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1620215211&Signature=Mt2Std3ijid8ntLwh3anT4DJ2t%2Ba4JzlrlBq29yXAqnwJlCLBWq3qenMV6Pr6QE3ZaWIiZfBFknZQ7EcMNJJtYS2l5GrnCRu68uT%2FPaZSSWoOA4PzOBFMAwnrffU8q%2FRogni4R0j%2B%2FFBGz%2FnrHw%2B2jLG%2F4%2F7gEY%2F7jhrNptqbVJFnLPxAwGWmlU9eGutByplN3A7gfSlbGk5TybkOPPzCG%2F%2B5TSvDmkPzkyNZ3wa84dbsjCLYiSvWFT0I%2FsnazgiVXUKWJUjF6AnQ3NNBYqWrWO2Kjc0s%2F63ZIhr06MvDdbe3C%2BJgCKiL8KV7vffGjyMhq4m1bGSdaD01VyZg0x19w%3D%3D&response-content-disposition=attachment%3B+filename%3Delo-merchant-category-recommendation.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.69.128, 64.233.183.128, 173.194.194.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.69.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 644665605 (615M) [application/zip]
Saving to: ‘el

In [ ]:
!unzip elo-merchant-category-recommendation.zip

Archive:  elo-merchant-category-recommendation.zip
  inflating: Data Dictionary.xlsx    
  inflating: Data_Dictionary.xlsx    
  inflating: historical_transactions.csv  
  inflating: merchants.csv           
  inflating: new_merchant_transactions.csv  
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


**loading data :**

In [ ]:
%%time
train_df, test_df, newmerchant_df, historical_df = loadData()

train_df = reduce_memory_usage(train_df)
test_df = reduce_memory_usage(test_df)
historical_df = reduce_memory_usage(historical_df)
newmerchant_df = reduce_memory_usage(newmerchant_df)

historical_df['category_3'].replace({'A':1, 'B':2,'C':3}, inplace=True)
newmerchant_df['category_3'].replace({'A':1, 'B':2,'C':3}, inplace=True)

Mem. usage decreased to  5.20 Mb (43.7% reduction)
Mem. usage decreased to  2.24 Mb (52.5% reduction)
Mem. usage decreased to 2026.75 Mb (34.8% reduction)
Mem. usage decreased to 136.66 Mb (34.8% reduction)
CPU times: user 4min 3s, sys: 11.7 s, total: 4min 15s
Wall time: 4min 14s


In [ ]:
%%time
train_data, test_data = baseline_features(train_df,test_df,historical_df,newmerchant_df)

target  = train_data[['card_id','target']]
target.set_index('card_id', inplace =True)

cards = pd.concat([train_data.drop(['target'] , axis= 1) , test_data] , axis = 0)

CPU times: user 2min 18s, sys: 4.52 s, total: 2min 23s
Wall time: 2min 17s


****Predicting Loyalty Score for a single card id of Test Data :****

In [ ]:
data_point = test_data.sample(1)['card_id'].to_list()

In [ ]:
%%time
predict_loyalty_score(data_point)

Fetching the transactional and card_id data
PreProcess the transactions data......
 - Imputing Missing values...
 - One Hot Encoding of variables...
Feature Engineering the transactions data.....
preprocess the Feature Engineered Data
Predicting the Loyalty Score.....
CPU times: user 1.67 s, sys: 43.7 ms, total: 1.72 s
Wall time: 1.67 s


,loyality_score
card_id,
C_ID_3c49e885d1,-0.519028


**Predicting Loyalty Score for a list of Test Card_id's :**

In [ ]:
data_set = test_data.sample(1000)['card_id'].to_list()
warnings.filterwarnings("ignore")

In [ ]:
%%time
predict_loyalty_score(data_set)

Fetching the transactional and card_id data
PreProcess the transactions data......
 - Imputing Missing values...
 - One Hot Encoding of variables...
Feature Engineering the transactions data.....
preprocess the Feature Engineered Data
Predicting the Loyalty Score.....
CPU times: user 5.4 s, sys: 144 ms, total: 5.55 s
Wall time: 5.31 s


,loyality_score
card_id,
C_ID_6a2c823e5f,-2.008991
C_ID_a749f3a780,-1.708625
C_ID_cc9840274c,-0.770120
C_ID_163ab1d052,-0.730276
C_ID_12bc7a41f0,-0.506894
...,...
C_ID_a49ad5697c,-1.189587
C_ID_88c7388492,-1.060245
C_ID_999cc0d894,-0.711917


In [ ]:
root_mean_squared_error(data_set, target.loc[target.index.isin(data_set)])

Fetching the transactional and card_id data
PreProcess the transactions data......
 - Imputing Missing values...
 - One Hot Encoding of variables...
Feature Engineering the transactions data.....
preprocess the Feature Engineered Data
Predicting the Loyalty Score.....
RMSE Score: 3.2518931751119102


**Note :** The Prediction time is 1.67 seconds. This latency can be reduced further by fetching the features of the transactions and card_id's from database.